In [1]:
import os, requests, json
import pandas as pd

In [2]:
BASE_URL = "https://www.caetrm.com/api/v1/measures/"
TOKEN    = os.getenv("CAE_TRM_TOKEN")   # pulled from the env

print(TOKEN)       # ← prints the actual token, or 'None' if it isn’t set

In [3]:
hdrs = {"Authorization": f"Token {TOKEN}"}
resp = requests.get(BASE_URL, headers=hdrs, timeout=30) #get measures

In [4]:
# --- basic error handling ---
resp.raise_for_status()        # throws for 4xx / 5xx; same as R’s stop_for_status
payload = resp.json()          # dict (already decoded from UTF-8)

print("Top-level keys:", payload.keys())
print("Total records:", payload["count"])

Top-level keys: dict_keys(['count', 'next', 'previous', 'results'])
Total records: 274


In [5]:
def fetch_all_measures(base_url=BASE_URL, headers=hdrs, verbose=False):
    url   = base_url         # may get overwritten with ?page=N
    frames = []

    while url:
        if verbose:
            print("Requesting:", url.split("?")[-1] or "<page 1>")
        r = requests.get(url, headers=headers, timeout=30)
        r.raise_for_status()
        data = r.json()

        frames.append(pd.json_normalize(data["results"]))
        url = data["next"]   # None when you reach the end

    return pd.concat(frames, ignore_index=True)

measures_df = fetch_all_measures(verbose=True)
print(measures_df.shape)      # rows x columns
measures_df.head()

Requesting: https://www.caetrm.com/api/v1/measures/
Requesting: limit=25&offset=25
Requesting: limit=25&offset=50
Requesting: limit=25&offset=75
Requesting: limit=25&offset=100
Requesting: limit=25&offset=125
Requesting: limit=25&offset=150
Requesting: limit=25&offset=175
Requesting: limit=25&offset=200
Requesting: limit=25&offset=225
Requesting: limit=25&offset=250
(274, 2)


,name,url
0,"Refrigerator, Residential",https://www.caetrm.com/api/v1/measures/SWAP001/
1,Refrigerator or Freezer Recycling,https://www.caetrm.com/api/v1/measures/SWAP002/
2,"Clothes Dryer, Residential",https://www.caetrm.com/api/v1/measures/SWAP003/
3,"Clothes Washer, Residential",https://www.caetrm.com/api/v1/measures/SWAP004/
4,"Ozone Laundry, Commercial",https://www.caetrm.com/api/v1/measures/SWAP005/


In [10]:
import requests
import pandas as pd

# ------------------------------------------------------------------
# 1.  Put your measure codes in a plain Python list
#     (equivalent to R's `as.list(Data_22$MeasureVersion)`).
# ------------------------------------------------------------------
codes = [
    "SWWH027/06","SWWH025/09",
    # add the rest of your codes here …
]

# ------------------------------------------------------------------
# 2.  Set up the authorization header once.
# ------------------------------------------------------------------
API_TOKEN = "yourtokenhere"
HEADERS    = {"Authorization": f"Token {API_TOKEN}"}

# ------------------------------------------------------------------
# 3.  Loop through the codes, call the endpoint, and build DataFrames.
# ------------------------------------------------------------------
output_data = {}              # Python dict ≈ R list()

for code in codes:
    url = f"https://www.caetrm.com/api/v1/measures/{code}/permutations"
    print(f"Fetching {url}")

    resp = requests.get(url, headers=HEADERS, timeout=30)
    resp.raise_for_status()   # throws if 4xx/5xx

    payload       = resp.json()              # dict
    rows          = payload["results"]       # list of records
    column_names  = payload["headers"]       # list of header strings

    df_measures = pd.DataFrame(rows, columns=column_names)
    output_data[code] = df                   # stash in dict

# ------------------------------------------------------------------
# 4.  (Optional)  Combine every dataframe into one tidy table
#     with `MeasureCode` as an index level.
# ------------------------------------------------------------------
# combined = pd.concat(output_data.values(),
#                      keys=output_data.keys(),
#                      names=["MeasureCode"])


Fetching https://www.caetrm.com/api/v1/measures/SWWH027/06/permutations
Fetching https://www.caetrm.com/api/v1/measures/SWWH025/09/permutations
